In [119]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors


In [120]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
user_vectors = pd.read_csv(r"D:\dataset\encoding\user_vectors.csv")
rating_vectors = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")

Label encode username column so that usernames can be retrieved after K-Nearest Neighbours calculation

In [243]:
encoder = LabelEncoder()
final_vectors = user_vectors.copy()
x = encoder.fit(final_vectors["username"])
#print(encoder.classes_)
username_enc = x.transform(final_vectors["username"])
username_enc

#adding encoded usernames to new dataframe
final_vectors["username"] = username_enc

#pd.set_option('display.max_rows', None)
final_arrays = final_vectors.values
#final_vectors.loc[final_vectors["username"]==7537]
#get username from number
#encoder.inverse_transform([7537])

Compute K-Nearest Neighbours

In [246]:
def neighbours():#function that returns a list of indexes of most similar users in "final_arrays" array
    user = np.array([7537,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
    nc = NearestNeighbors(n_neighbors = 10, metric="cosine") #euclidean
    
    train = nc.fit(final_arrays)
    n = train.kneighbors([user], return_distance=False)
    return n

sim_names = []
neighbours = neighbours()
for i in neighbours[0]: #loop through nested array
    #print(final_arrays[i][0])
    name = x.inverse_transform([final_arrays[i][0]])
    sim_names.append(name)

#build new dataframe of neighbours
neighbour_frame = pd.DataFrame()
for i in users.columns:
    neighbour_frame[i] = None

for name in sim_names:
    user_row = users.loc[users["username"]==name[0]]
    neighbour_frame = pd.concat([neighbour_frame, user_row])
neighbour_frame


#(final_vectors.columns)    

,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Action', 'Shounen']",Classic
68,502880,lila94,89,Active,Female,1994,4204,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
416,2120443,hana_haruka,110,Active,Female,1993,4062,Millenials,"['Action', 'Shounen', 'Comedy']",Classic
3962,71147,gosakurago,86,Active,Female,1994,3329,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
3132,487038,gorankudasai,144,Active,Female,1992,3513,Millenials,"['Action', 'Shounen', 'Comedy']",Classic
3507,1263323,mzpasaway13,86,Active,Female,1994,4150,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
3037,22536,nawara,49,Active,Female,1984,3257,Millenials,"['Comedy', 'Shounen', 'Action']",Classic
4223,440913,ninaka,272,Active,Female,1992,5788,Millenials,"['Comedy', 'Shounen', 'Action']",Classic
6068,446074,shirah0shi,137,Active,Female,1991,4640,Millenials,"['Action', 'Comedy', 'Shounen']",Classic
4301,158358,chibi_miyavi,137,Active,Female,1992,3561,Millenials,"['Comedy', 'Shounen', 'Action']",Classic


Function that converts user information from website into vector